<a href="https://colab.research.google.com/github/VijayaBhargavi198/5731Assignments/blob/master/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
from zipfile import ZipFile 
file_name = '/content/Assignment four data Yelp (question 1 and 2).zip'
with ZipFile(file_name, 'r') as zip:  
    # extracting all the files 
    zip.extractall() 
    print('Extraction of files complete!')

Extraction of files complete!


In [133]:
import os,json
import pandas as pd
arr = os.listdir('/content/Assignment four data Yelp (question 1 and 2)')
reviews = []
ratings = []
for file in arr:
  with open('/content/Assignment four data Yelp (question 1 and 2)/'+file,encoding = "utf-8") as i:
    data = json.load(i)
    for j in data:
      reviews.append(j['text'])
      ratings.append(j['stars']) 
input_df = pd.DataFrame(reviews, columns=["review"])
input_df["ratings"]=ratings
input_df.head()

,review,ratings
0,"Awesome service and fantastic food, the wait s...",5.0
1,Seemed pretty pricey to me for what you got......,2.0
2,We TRIED to go to the buffet on Tuesday eve fo...,1.0
3,Came here for lunch on my 20th birthday and th...,5.0
4,"Amazing treats - I love their cinnamon roll, c...",5.0


In [134]:
# Preprocessing data to be used in text modelling
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import pandas as pd
from textblob import Word
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words=stopwords.words("english")
# cleaning the text data
input_df['cleaned text review'] = input_df['review'].str.replace(r"\W", " ").str.strip()# 1.To remove special characters and punctuations
input_df['cleaned text review'] = input_df['cleaned text review'].str.replace(r'\d+',"") #2.To remove Numbers
input_df['cleaned text review'] = input_df['cleaned text review'].apply(lambda x: " ".join(x.lower() for x in x.split())) #convert the whole text into lower case
input_df['cleaned text review'] = input_df['cleaned text review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words)) #remove all stop words
input_df.head() # dataframe column that would be used for further process

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,ratings,cleaned text review
0,"Awesome service and fantastic food, the wait s...",5.0,awesome service fantastic food wait staff frie...
1,Seemed pretty pricey to me for what you got......,2.0,seemed pretty pricey got burgers tasted pretty...
2,We TRIED to go to the buffet on Tuesday eve fo...,1.0,tried go buffet tuesday eve dinner found close...
3,Came here for lunch on my 20th birthday and th...,5.0,came lunch th birthday gave us free dessert si...
4,"Amazing treats - I love their cinnamon roll, c...",5.0,amazing treats love cinnamon roll cupcakes ice...


In [ ]:
! pip install biterm
! pip install pyLDAvis

In [ ]:
import numpy as np
import pyLDAvis
from biterm.btm import oBTM 
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary # helper functions
# vectorize texts
texts = input_df['cleaned text review'].head(1000).values
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(texts).toarray()
vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X)
btm = oBTM(num_topics=20, V=vocab)
print("Train Online BTM:")
topics = btm.fit_transform(biterms, iterations=20)

Train Online BTM:




















  0%|          | 0/20 [00:00<?, ?it/s]

















  5%|▌         | 1/20 [09:17<2:56:37, 557.78s/it]

In [ ]:

    print("reviews realted to each topic are")
    for i in range(len(texts)):
        print("{} (topic: {})".format(texts[i], topics[i].argmax()))

# **Question 2: Yelp Review Sentiment Analysis**
 Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: 80% data for training and 20% data for testing.
 Answer the following questions:

(1) Features used for sentiment classification and explain why you select these features (tf-idf, sentiment lexicon, word2vec, etc). Considering achieve the best performance as you can.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively.

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.|

In [ ]:
def sentiment(x):
    if x == 5:
        return 'Very positive'
    if x == 4:
        return 'Positive'
    if x == 3:
        return 'Neutral'
    if x == 2:
        return 'Negative'
    if x == 1:
        return 'Very negative'
    
input_df['sentiment'] = input_df['ratings'].apply(lambda x: sentiment(x))
input_df.head(5)

,review,ratings,cleaned text review,sentiment
0,"Awesome service and fantastic food, the wait s...",5.0,awesome service fantastic food wait staff frie...,Very positive
1,Seemed pretty pricey to me for what you got......,2.0,seemed pretty pricey got burgers tasted pretty...,Negative
2,We TRIED to go to the buffet on Tuesday eve fo...,1.0,tried go buffet tuesday eve dinner found close...,Very negative
3,Came here for lunch on my 20th birthday and th...,5.0,came lunch th birthday gave us free dessert si...,Very positive
4,"Amazing treats - I love their cinnamon roll, c...",5.0,amazing treats love cinnamon roll cupcakes ice...,Very positive


In [ ]:
#SVM 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
train, test = sklearn.model_selection.train_test_split(input_df, train_size=0.8, test_size=0.2)
svm_pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier( max_iter=100,learning_rate='optimal'))])
svm = svm_pipeline.fit(train['cleaned text review'], train['sentiment'])
test['SVM_predicted sentiment'] = svm.predict(test['cleaned text review'])
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print("Results for first supervised learning algorithm, SVM: "'\n')
print(test[['review', 'sentiment', 'SVM_predicted sentiment']].head(5))
svm_accuracy = accuracy_score(test['sentiment'], test['SVM_predicted sentiment'])*100
svm_precision = precision_score(test['sentiment'], test['SVM_predicted sentiment'], average='macro')
svm_recall = recall_score(test['sentiment'], test['SVM_predicted sentiment'], average='macro')
svm_f1 = f1_score(test['sentiment'], test['SVM_predicted sentiment'], average='macro')

print('\n', 'The accuracy of the TFIDF-based SVM sentiment identification is:', svm_accuracy)
print('The precision score of the TFIDF-based SVM sentiment identification is:', svm_precision)
print('The recall score of the TFIDF-based SVM sentiment identification is:', svm_recall)
print('The f1-score of the TFIDF-based SVM sentiment identification is:', svm_f1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Results for first supervised learning algorithm, SVM: 

                                                   review  ... SVM_predicted sentiment
218848  I am a fan of Beer Express because of their se...  ...           Very positive
398884  I've given this place numerous tries because t...  ...           Very negative
313014  This Place is the best!\n\nHigh quality meats....  ...           Very positive
145799  Went to Euro nails on Friday after my sister f...  ...           Very positive
196443  We came here last night for a late supper and ...  ...                Positive

[5 rows x 3 columns]

 The accuracy of the TFIDF-based SVM sentiment identification is: 61.925
The precision score of the TFIDF-based SVM sentiment identification is: 0.5331333804040859
The recall score of the TFIDF-based SVM sentiment identification is: 0.4606307465296361
The f1-score of the TFIDF-based SVM sentiment identification is: 0.4382381269146757


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_pipeline = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB())])
nb = nb_pipeline.fit(train['cleaned text review'], train['sentiment'])
test['NB_predicted sentiment'] = nb.predict(test['cleaned text review'])
print("Results for second supervised learning algorithm,Naive Bayes :", '\n')
print(test[['review', 'sentiment', 'NB_predicted sentiment']].head(5))
NB_accuracy = accuracy_score(test['sentiment'], test['NB_predicted sentiment'])*100
NB_f1 = f1_score(test['sentiment'], test['NB_predicted sentiment'], average='macro')
NB_precision = precision_score(test['sentiment'], test['NB_predicted sentiment'], average='macro')
NB_recall = recall_score(test['sentiment'], test['NB_predicted sentiment'], average='macro')

print('The accuracy of the TFIDF-based Naive Bayes sentiment identification is:', NB_accuracy)
print('The precision score of the TFIDF-based Naive Bayes sentiment identification is:', NB_precision)
print('The recall score of the TFIDF-based Naive Bayes sentiment identification is:', NB_recall)
print('The f1-score of the TFIDF-based Naive Bayes sentiment identification is:', NB_f1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Results for second supervised learning algorithm,Naive Bayes : 

                                                   review  ... NB_predicted sentiment
218848  I am a fan of Beer Express because of their se...  ...          Very positive
398884  I've given this place numerous tries because t...  ...          Very positive
313014  This Place is the best!\n\nHigh quality meats....  ...          Very positive
145799  Went to Euro nails on Friday after my sister f...  ...          Very positive
196443  We came here last night for a late supper and ...  ...          Very positive

[5 rows x 3 columns]
The accuracy of the TFIDF-based Naive Bayes sentiment identification is: 56.774
The precision score of the TFIDF-based Naive Bayes sentiment identification is: 0.46440150565373395
The recall score of the TFIDF-based Naive Bayes sentiment identification is: 0.37244025511316664
The f1-score of the TFIDF-based Naive Bayes sentiment identification is: 0.3425739130327179
